In [ ]:
from IPython.display import display, Javascript, Markdown, clear_output, HTML
import ipywidgets as ipw
import json
import subprocess
import atexit
import os
import shutil
from src import widgets, utils

In [ ]:
CONFIG_FILENAME = "schema/config.json"
CONFIG = utils.read_json(CONFIG_FILENAME)
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis_aiida()

In [ ]:
if OPENBIS_SESSION:
    # This is to adjust tab title length
    display(
        HTML(
            '''
                <style>
                /* Target tab labels inside ipywidgets */
                .jp-Notebook .widget-tab .p-TabBar-tab,
                .widget-tab .p-TabBar-tab {
                    max-width: none !important;
                    min-width: 200px !important;   /* You can adjust this */
                    white-space: normal !important;
                    overflow-wrap: break-word !important;
                    word-break: break-word !important;
                }
                </style>
            '''
        )
    )
    
    increase_buttons_size = HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
    tab_contents = ["Start directory monitoring", "Directories being monitorised"]
    running_watchdogs_vbox = widgets.RunningMeasurementWatchdogsWidget(OPENBIS_SESSION, SESSION_DATA)
    measurement_uploader_vbox = widgets.GenerateMeasurementsWatchdogWidget(OPENBIS_SESSION, SESSION_DATA, running_watchdogs_vbox)
    
    sample_measurement_tabs = ipw.Tab(
        children = [measurement_uploader_vbox, running_watchdogs_vbox],
        layout = ipw.Layout(width='auto')
    )
    for i,e in enumerate(tab_contents):
        sample_measurement_tabs.set_title(i, e)

home_button = ipw.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)

In [ ]:
# # Keep track of the process globally so we can terminate it later
watchdog_processes = []

def generate_watchdog(b):
    global watchdog_processes

    experiment_id = sample_measurement_vbox.select_experiment_widget.experiment_dropdown.value
    if experiment_id == "-1":
        return
    
    sample_id = sample_measurement_vbox.select_sample_widget.sample_dropdown.value
    if sample_id == "-1":
        return
    
    sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_id)
    sample_name = sample_object.props["name"]

    instrument_id = sample_measurement_vbox.select_instrument_widget.instrument_dropdown.value
    if instrument_id == "-1":
        return

    measurement_session_object = utils.create_openbis_object(
        OPENBIS_SESSION,
        type = "MEASUREMENT_SESSION",
        collection = experiment_id,
        parents = [sample_id, instrument_id],
        props = {"name": f"Measurement Session on Sample {sample_name}", "default_object_view": "IMAGING_GALLERY_VIEW"}
    )
    measurement_session_id = measurement_session_object.permId
    measurements_directory = f"/home/jovyan/aiida-openbis/Notebooks/openbis_app/measurement_data_{measurement_session_id}"
    watchdog_file = f"/home/jovyan/aiida-openbis/Notebooks/openbis_app/src/measurements_uploader.py"
    if os.path.exists(measurements_directory) == False:
        os.makedirs(measurements_directory)
        shutil.copy(watchdog_file, measurements_directory)

    watchdog_process = subprocess.Popen(
        [
            "python", 
            f"{measurements_directory}/measurements_uploader.py", 
            "--openbis_url", SESSION_DATA["url"],
            "--openbis_token", SESSION_DATA["token"],
            "--measurement_session_id", measurement_session_object.permId,
            "--data_folder", measurements_directory
        ]
    )
    print("Watchdog process started with PID:", watchdog_process.pid)
    
    watchdog_processes.append(watchdog_process)

def cleanup_watchdog():
    global watchdog_processes
    if watchdog_processes:
        for process in watchdog_processes:
            print(f"Terminating watchdog process with PID: {process.pid}")
            process.terminate()
            watchdog_processes = []

# Ensure process is killed on notebook shutdown / kernel restart
atexit.register(cleanup_watchdog)

def close_notebook(b):
    display(Javascript(data = 'window.location.replace("home.ipynb")'))

# Measurement Uploader

In [ ]:
if OPENBIS_SESSION:
    display(sample_measurement_tabs)
    display(increase_buttons_size)

home_button.on_click(close_notebook)
display(home_button)